In [1]:
# !pip install qiskit-nature

In [2]:
from math import pi
import json
from pathlib import Path
from math import sqrt
from typing import *

import numpy as np
from dotenv import load_dotenv
import rustworkx as rx
import networkx as nx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.problems import LatticeModelProblem
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Sampler
from qiskit.quantum_info import Pauli
from qiskit.result import QuasiDistribution
from qiskit_algorithms import QAOA as QAOASolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from process_bigraph import Process, Step
from process_bigraph import pp

from bio_bundles import registrar


load_dotenv("../.env")

True

In [3]:
from process_bigraph import Process, Step


class RQAOA(Process):
    pass


class VariableReduction(Process):
    """Run reduce_variables() workflow here"""
    pass


class VariableChecker(Step):
    """takes N as input state and checks if N == Ncrit"""
    pass


from qiskit.quantum_info import SparsePauliOp
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        paulis = ["I"] * len(graph)
        paulis[edge[0]], paulis[edge[1]] = "Z", "Z"

        weight = graph.get_edge_data(edge[0], edge[1])

        pauli_list.append(("".join(paulis)[::-1], weight))

    return pauli_list


class MolecularQAOA(Process):
    config_schema = {
        'nuclei': 'list[list[float]]'
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # position vectors of all nuclei involved in solving
        self.R = self.config['nuclei']

In [4]:
# from bio_bundles import registrar
# CORE = registrar.core
# 
# rqaoa = RQAOA(
#     config={
#         'n_nodes': 5,
#         'edge_list': [(0, 1, 1.0), (0, 2, 1.0), (0, 4, 1.0), (1, 2, 1.0), (2, 3, 1.0), (3, 4, 1.0)] 
#     },
#     core=CORE
# )

"""R is an array of arrays of shape: (n, 3)
    where n is the number of nuclei involved in the hamiltonian, and 
    3 represent position vector dimesions (x, y, z) for each n
"""

'R is an array of arrays of shape: (n, 3)\n    where n is the number of nuclei involved in the hamiltonian, and \n    3 represent position vector dimesions (x, y, z) for each n\n'

In [37]:
class GroverProcess(Process):
    config_schema = {
        "target": "string"
    }

    def __init__(self, config, core):
        super().__init__(config, core)





In [38]:
CORE = registrar.core

registrar.register_process("qaoa", QAOA, verbose=True)

Successfully registered <class '__main__.QAOA'> to address: qaoa
